##### 해시법
- 해시법 : '데이터를 저장할 위치 = 인덱스'를 간단한 연산으로 구하는 것
- 해시 함수 : 키를 해시값으로 변환하는 과정
- 버킷 : 해시테이블에서 만들어진 원소

##### 해시 충돌
키와 해시값은 일반적으로 다대일(n:1) 관계
- 충돌 : 저장할 버킷이 중복되는 현상
- 충돌 해결 방법 
    - 체인법 : 해시값이 같은 원소를 연결 리스트로 관리
    - 오픈 주소법 : 빈 버킷을 찾을 때까지 해시를 반복

##### 체인법
- 체인법(오픈 해시법) : 해시값이 같은 데이터를 체인 모양의 연결 리스트로 연결하는 방법
###### 3-5[A]

In [2]:
# 체인법으로 해시 함수 구현하기

from __future__ import annotations
from typing import Any, Type
import hashlib

class Node:
    """해시를 구성하는 노드"""

    def __init__(self, key: Any, value: Any, next: Node) -> None:
        """초기화"""
        self.key = key      # 키
        self.value = value  # 값
        self.next = next    # 뒤쪽 노드를 참조

###### 3-5[B, C, D]

In [3]:
class ChainedHash:
    """체인법으로 해시 클래스 구현"""

    def __init__(self, capacity: int) -> None:
        """초기화"""
        self.capacity = capacity        # 해시 테이블의 크기를 지정
        self.table = [None] * self.capacity # 해시테이블(리스트)를 선언

    def hash_value(self, key:Any) -> int:
        """해시값을 구함"""
        if isinstance(key, int):
            return key % self.capacity
        return (int(hashlib.sha256(str(key).encode()).hexdigest(), 16) % self.capacity)

    def search(self, key: Any) -> Any:
        """키가 key인 원소를 검색하여 값을 반환"""
        hash = self.hash_value(key)         # 검색하는 키의 해시값
        p = self.table[hash]                # 노드를 주목

        while p is not None:
            if p.key == key:
                return p.value              # 검색 성공
            p = p.next                      # 뒤쪽 노드를 주목

        return None                         # 검색 실패

    def add(self, key: Any, value: Any) -> bool:
        """키가 key이고 값이 value인 원소를 추가"""
        hash = self.hash_value(key)         # 추가하는 key의 해시값
        p = self.table[hash]                # 노드를 주목

        while p is not None:
            if p.key == key:
                return False                # 추가 실패
            p = p.next                      # 뒤쪽 노드를 주목

        temp = Node(key, value, self.table[hash])   
        self.table[hash] = temp             # 노드를 추가
        return True                         # 추가 성공

    def remove(self, key: Any) -> bool:
        """키가 key인 원소를 삭제"""
        hash = self.hash_value(key)     # 삭제할 key의 hash값
        p = self.table[hash]            # 노드를 주목
        pp = None                       # 바로 앞의 노드를 주목

        while p is not None:
            if p.key == key:            # key를 발견하면 아래를 실행
                if pp is None:
                    self.table[hash] = p.next
                else:
                    pp.next = p.next
                return True     # key 삭제 성공

            pp = p
            p = p.next                  # 뒤쪽 노드를 주목
        return False                    # 삭제 실패(key가 존재하지 않음)

    def dump(self) -> None:
        """해시 테이블을 덤프"""
        for i in range(self.capacity):
            p = self.table[i]
            print (i, end='')
            while p is not None:
                print (f'  -> {p.key} ({p.value})', end = '')
                p = p.next
            print ()

- sha256 알고리즘 : hashlib 모듈에서 제공하는 sha256은 RSA의 FIPS 알고리즘을 바탕으로 하며, 주어진 바이트(byte) 문자열의 해시값을 구하는 해시 알고리즘의 생성자(xonstructor)입니다. hashlib모듈은 sha256 외에도 MD5 알고리즘인 md5 등 다양한 해시 알고리즘을 제공합니다.  
  
- encode() 함수 : hashlib.sha256에는 바이트 문자열의 인수를 전달해야합니다. 그래서 key를 str형 문자열로 변환한 뒤 그 문자열을 encode() 함수에 전달하여 바이트 문자열을 생성합니다.

- hexdigest() 함수 : sha256 알고리즘에서 해시값을 16진수 문자열로 꺼냅니다.

- int() 함수 : hexdigest() 함수로 꺼낸 문자열을 16진수 문자열로 하는 int형으로 변환합니다.

###### 3-6

In [1]:
from enum import Enum
from chained_hash import ChainedHash

Menu = Enum('Menu', ['추가', '삭제', '검색', '덤프', '종료'])   # 메뉴를 선언

def select_menu() -> Menu:
    """메뉴 선택"""
    s = [f'({m.value}){m.name}' for m in Menu]
    while True:
        print (*s, sep = '  ', end='')
        n = int(input(': '))
        if 1 <= n <= len(Menu):
            return Menu(n)

hash = ChainedHash(13)          # 크기가 13인 해시 테이블을 생성

while True:
    menu = select_menu()        # 메뉴 선택

    if menu == Menu.추가:       # 추가
        key = int(input('추가할 키를 입력하세요 : '))
        val = input('추가할 값을 입력하세요 : ')
        if not hash.add(key, val):
            print ('추가를 실패했습니다.')

    elif menu == Menu.삭제:     # 삭제
        key = int(input('삭제할 키를 입력하세요 : '))
        if not hash.remove(key):
            print ('삭제를 실패했습니다.')

    elif menu == Menu.검색:     # 검색
        key = int(input('검색할 키를 입력하세요.'))
        t = hash.search(key)
        if t is not None:
            print (f'검색한 키가 갖는 값은 {t}입니다')
        else:
            print ('검색할 데이터가 없습니다')

    elif menu == Menu.덤프:     # 덤프
        hash.dump()

    else:                       # 종료
        break

(1)추가  (2)삭제  (3)검색  (4)덤프  (5)종료(1)추가  (2)삭제  (3)검색  (4)덤프  (5)종료(1)추가  (2)삭제  (3)검색  (4)덤프  (5)종료(1)추가  (2)삭제  (3)검색  (4)덤프  (5)종료(1)추가  (2)삭제  (3)검색  (4)덤프  (5)종료(1)추가  (2)삭제  (3)검색  (4)덤프  (5)종료검색한 키가 갖는 값은 동혁입니다
(1)추가  (2)삭제  (3)검색  (4)덤프  (5)종료0
1  -> 14 (민서)  -> 1 (수연)
2
3
4
5  -> 5 (동혁)
6
7
8
9
10  -> 10 (예지)
11
12  -> 12 (원준)
(1)추가  (2)삭제  (3)검색  (4)덤프  (5)종료(1)추가  (2)삭제  (3)검색  (4)덤프  (5)종료0
1  -> 1 (수연)
2
3
4
5  -> 5 (동혁)
6
7
8
9
10  -> 10 (예지)
11
12  -> 12 (원준)
(1)추가  (2)삭제  (3)검색  (4)덤프  (5)종료

##### 오픈주소법
###### 3-7

In [3]:
# 오픈 주소법으로 해시 함수 구현하기

from __future__ import annotations
from typing import Any, Type
from enum import Enum
import hashlib

# 버킷의 속성
class Status(Enum):
    OCCUPIED = 0    # 데이터를 저장
    EMPTY = 1       # 비어있음
    DELETED = 2     # 삭제 완료

class Bucket:
    """해시를 구성하는 버킷"""

    def __init__(self, key: Any = None, value: Any = None, stat: Status = Status.EMPTY) -> None:
        """초기화"""
        self.key = key              # 키
        self.value = value          # 값
        self.stat = stat            # 속성

    def set(self, key: Any, value: Any, stat: Status) -> None:
        """모든 필드에 값을 설정"""   
        self.key = key              # 키
        self.value = value          # 값
        self.stat = stat            # 속성

    def set_status(self, stat: Status) -> None:
        """속성을 설정"""
        self.stat = stat

class OpenHash:
    """오픈 주소법으로 구현하는 해시 클래스"""

    def __init__(self, capacity: int) -> None:
        """초기화"""
        self.capacity = capacity                    # 해시 테이블의 크기를 지정
        self.table = [Bucket()] * self.capacity     # 해시 테이블

    def hash_value(self, key: Any) -> int:
        """해시값을 구함"""
        if isinstance(key, int):
            return key % self.capacity
        return (int(hashlib.md5(str(key).encode()).hexdigest(), 16) % self.capacity)

    def rehash_value(self, key: Any) -> int:
        """재해시값을 구함"""
        return (self.hash_value(key) + 1) % self.capacity

    def search_node(self, key: Any) -> Any:
        """키가 key인 버킷을 검색"""
        hash = self.hash_value(key)         # 검색하는 키의 해시값
        p = self.table[hash]                # 버킷을 주목

        for i in range(self.capacity):
            if p.stat == Status.EMPTY:
                break
            elif p.stat == Status.OCCUPIED and p.key == key:
                return p
            hash = self.rehash_value(hash)  # 재해시
            p = self.table[hash]
        return None

    def search(self, key: Any) -> Any:
        """키가 key인 원소를 검색하여 값을 반환"""
        p = self.search_node(key)
        if p is not None:
            return p.value                  # 검색 성공
        else:
            return None                     # 검색 실패

    def add(self, key: Any, value: Any) -> bool:
        """키가 key이고 값이 value인 원소를 추가"""
        if self.search(key) is not None:
            return False                    # 이미 등록된 키

        hash = self.hash_value(key)         # 추가하는 키의 해시값
        p = self.table[hash]                # 버킷을 주목
        for i in range(self.capacity):
            if p.stat == Status.EMPTY or p.sata == Status.DELETED:
                self.table[hash] = Bucket(key, value, Status.OCCUPIED)
                return True

            hash = self.rehash_value(hash)  # 재해시
            p = self.table[hash]
        return False                        # 해시 테이블이 가득 참

    def remove(self, key: Any) -> int:
        """키가 key인 원소를 삭제"""
        p = self.search_node(key)           # 버킷을 주목
        if p is None:
            return False                    # 이 키는 등록되어있지 않음
        p.set_status(Status.DELETED)
        return True

    def dump(self) -> None:
        """해시 테이블을 덤프"""
        for i in range(self.capacity):
            print (f'{i: 2} ', end='')
            if self.table[i].stat == Status.OCCUPIED:
                print (f'{self.table[i].key} ({self.table[i].value})')
            elif self.table[i].stat == Status.EMPTY:
                print ('-- 미등록 --')
            elif self.table[i].stat == Status.DELETED:
                print ('-- 삭제 완료 --')



###### 3-8

In [1]:
# 오픈 주소법을 구현하는 해시 클래스 Openhash 사용

from enum import Enum
from open_hash import OpenHash

Menu = Enum('Menu', ['추가', '삭제', '검색', '덤프', '종료'])   # 메뉴를 선언

def select_menu() -> Menu:
    """메뉴 선택"""
    s = [f'({m.value}){m.name}' for m in Menu]
    while True:
        print (*s, sep = '  ', end = '')
        n = int(input(':  '))
        if 1 <= n <= len(Menu):
            return Menu(n)

hash = OpenHash(13)     # 크기가 13인 해시 테이블 생성

while True:
    menu = select_menu()        # 메뉴 선택

    if menu == Menu.추가:       # 추가
        key = int(input('추가할 키를 입력하세요 : '))
        val = input('추가할 값을 입력하세요 : ')
        if not hash.add(key, val):
            print ('추가를 실패했습니다!')

    elif menu == Menu.삭제:     # 삭제
        key = int(input('삭제할 키를 입력하세요 : '))
        if not hash.remove(key):
            print ('삭제를 실패했습니다.')

    elif menu == Menu.검색:     # 검색
        key = int(input('검색할 키를 입력하세요'))
        t = hash.search(key)
        if t is not None:
            print (f'검색한 키가 갖는 값은 {t}입니다.')
        else:
            print ('검색할 데이터가 없습니다.')

    elif menu == Menu.덤프:     # 덤프
        hash.dump()

    else:                       # 종료
        break

(1)추가  (2)삭제  (3)검색  (4)덤프  (5)종료(1)추가  (2)삭제  (3)검색  (4)덤프  (5)종료(1)추가  (2)삭제  (3)검색  (4)덤프  (5)종료(1)추가  (2)삭제  (3)검색  (4)덤프  (5)종료(1)추가  (2)삭제  (3)검색  (4)덤프  (5)종료(1)추가  (2)삭제  (3)검색  (4)덤프  (5)종료검색한 키가 갖는 값은 동혁입니다.
(1)추가  (2)삭제  (3)검색  (4)덤프  (5)종료 0 -- 미등록 --
 1 1 (수연)
 2 14 (민서)
 3 -- 미등록 --
 4 -- 미등록 --
 5 5 (동혁)
 6 -- 미등록 --
 7 -- 미등록 --
 8 -- 미등록 --
 9 -- 미등록 --
 10 10 (예지)
 11 -- 미등록 --
 12 12 (원준)
(1)추가  (2)삭제  (3)검색  (4)덤프  (5)종료 0 -- 미등록 --
 1 1 (수연)
 2 14 (민서)
 3 -- 미등록 --
 4 -- 미등록 --
 5 5 (동혁)
 6 -- 미등록 --
 7 -- 미등록 --
 8 -- 미등록 --
 9 -- 미등록 --
 10 10 (예지)
 11 -- 미등록 --
 12 12 (원준)
(1)추가  (2)삭제  (3)검색  (4)덤프  (5)종료(1)추가  (2)삭제  (3)검색  (4)덤프  (5)종료 0 -- 미등록 --
 1 1 (수연)
 2 -- 삭제 완료 --
 3 -- 미등록 --
 4 -- 미등록 --
 5 5 (동혁)
 6 -- 미등록 --
 7 -- 미등록 --
 8 -- 미등록 --
 9 -- 미등록 --
 10 10 (예지)
 11 -- 미등록 --
 12 12 (원준)
(1)추가  (2)삭제  (3)검색  (4)덤프  (5)종료